In [1]:
from VC_collections import Collection
from VC_collections.Collection import connect_to_google_drive
from VC_collections.AuthorityFiles import Authority_instance
from preprocess_1 import update_df_in_gdrive
import gspread
import pandas as pd

In [2]:
cred = Collection.get_google_drive_credentials()
client = gspread.authorize(cred)

# Create One big Dataframe of all tables

In [3]:
catalog_files = {
    "Architect": "1EhFxuphaOu-S8rZy_7O8OuLk3BMH_rAXjOTqDg5yMvY",
    "Dance": "1WcqUNkDhNbDyTdYLU_S3TbVkjZ6G03m36_9FqvBgaSc",
    "Design": "1M-vegxBcBI7J5hZvoqLkQQ7ATxMmfyyrFpAX4-vR7FM",
    "Theater": "1goBGosdUcCLzmdxIuF1SRYgk_mIT-eIsCJgdvd69rCw",
}


def create_df_from_gspread(client, branch):
    spread = client.open_by_key(catalog_files[branch])
    sheet = spread.worksheet("Master Catalog")
    df = pd.DataFrame(sheet.get_all_values())
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header
    df.set_index("MMS id", inplace=True)
    return df

In [33]:
df_Architect = create_df_from_gspread(client, "Architect")

In [5]:
df_Dance = create_df_from_gspread(client, "Dance")

In [6]:
df_Design = create_df_from_gspread(client, "Design")

In [7]:
df_Theater = create_df_from_gspread(client, "Theater")

In [8]:
master_catalog = pd.concat([df_Architect, df_Dance, df_Design, df_Theater])

In [9]:
master_catalog.index = master_catalog.index.astype(str)

In [10]:
master_catalog.head()

,ארכיון,סימול,סימול אב,ברקוד,סימול מקורי,רמת תיאור,מספר מיכל,קוד תיק ארכיון,כותרת,כותרת אנגלית,...,חומרים קשורים,תאריך פקיעת החיסיון,תקציר,קרדיט דימוי1,קרדיט דימוי2,מספר מערכת דימוי מייצג,שם הקובץ בתיקיה של הפריט המייצג,קרדיט עברית,קרדיט אנגלית,סוג הארכיון
MMS id,,,,,,,,,,,,,,,,,,,,,
990049153800205171,ArBe,ArBe,,,,Fonds Record,,,אוסף דוד בסט,,...,,,,,,,,NaN,NaN,NaN
990049153810205171,ArBe,ArBe-001,ArBe,,,Sub-Fonds Record,,,תכניות ושרטוטים מגולגלים,Rolled-up floor plans and sketches,...,,,,,,,,NaN,NaN,NaN
990049153820205171,ArBe,ArBe-001-001,ArBe-001,202949,,Series Record,,,תכניות ושרטוטים - שכונת נוה דוד - דימונה 1 מתוך 2,Floor plans and sketches - Neve David neighbor...,...,,,,,,,,NaN,NaN,NaN
990049153830205171,ArBe,ArBe-001-002,ArBe-001,3788,,Series Record,,,תכניות ושרטוטים - שכונת נוה דוד - דימונה 2 מתוך 2,Floor plans and sketches - Neve David neighbor...,...,,,,,,,,NaN,NaN,NaN
990049153840205171,ArBe,ArBe-001-003,ArBe-001,202965,,Series Record,,,תכניות ושרטוטים - שיקום בתים - קונטרה 1 מתוך 2,Floor plans and sketches - Housing restoration...,...,,,,,,,,NaN,NaN,NaN


In [11]:
len(sorted(master_catalog["ארכיון"].unique()))

102

# get bad records

In [34]:
df_repairs = pd.ExcelFile("Data/copyright_repairs_18102021.xlsx").parse("Sheet1").set_index("001")

In [35]:
df_repairs.head(2)

,245$a,520$a,655$a,כנראה לפתיחה תוכניות ושרטוטים מדיניות חדשה,093$a,093$c,093$d,פרטיות,952$f,008$7:4,...,952$e,915,597$a,597$b,999$a,906$a,907$i,907$c,907$k.1,700
001,,,,,,,,,,,,,,,,,,,,,
990049383680205171,"תצלום תכנית - בי""ח מאיר 3 מתוך 16.",תצלום - תכנית קומה ראשונה של בית החולים מאיר,copies (documents);photographs,כן,NaN,ArZa-001-001-013,ארכיון זרחי אדריכלים,NaN,אין מגבלת פרטיות,1935,...,NaN,\\$aהסכם זרחי אדריכלים 10032017,ארכיון זרחי אדריכלים זמין דיגיטלית באדיבות המש...,NaN,PHOTOGRAPH;ARCHIVE,VIS;DIG,2,IE52225608,AR_IP_RANGE,NaN
990049383690205171,"תצלום תכנית - בי""ח מאיר 4 מתוך 16.",תצלום - תכנית קומה שנייה של בית החולים מאיר,copies (documents);photographs,כן,NaN,ArZa-001-001-014,ארכיון זרחי אדריכלים,NaN,אין מגבלת פרטיות,1935,...,NaN,\\$aהסכם זרחי אדריכלים 10032017,ארכיון זרחי אדריכלים זמין דיגיטלית באדיבות המש...,NaN,PHOTOGRAPH;ARCHIVE,VIS;DIG,2,IE52225596,AR_IP_RANGE,NaN


In [57]:
df_Architect.head(2)

,ארכיון,סימול,סימול אב,ברקוד,סימול מקורי,רמת תיאור,מספר מיכל,קוד תיק ארכיון,כותרת,כותרת אנגלית,...,אוסף פתוח,ביבליוגרפיה ומקורות מידע,מיקום פיזי,חומרים קשורים,תאריך פקיעת החיסיון,תקציר,קרדיט דימוי1,קרדיט דימוי2,מספר מערכת דימוי מייצג,שם הקובץ בתיקיה של הפריט המייצג
MMS id,,,,,,,,,,,,,,,,,,,,,
990049153800205171,ArBe,ArBe,,,,Fonds Record,,,אוסף דוד בסט,,...,כן,"Best, David. Architecture and Urban Planning -...","משרד האדריכלים 'בסט אדריכלים ומתככני ערים בע""מ'",,,,,,,
990049153810205171,ArBe,ArBe-001,ArBe,,,Sub-Fonds Record,,,תכניות ושרטוטים מגולגלים,Rolled-up floor plans and sketches,...,,,,,,,,,,


In [36]:

df_Architect.index = df_Architect.index.astype(str)

In [37]:
df_repairs.index = df_repairs.index.rename("MMS id")
    

In [50]:
df_repairs.index = df_repairs.index.astype(str)

In [51]:
type(df_repairs.index[0])


str

In [52]:
df_Architect.loc[df_repairs.index[0], ["תאריך מנורמל מוקדם", "תאריך מנורמל מאוחר", "סימול"]]

0
תאריך מנורמל מוקדם                1935
תאריך מנורמל מאוחר                2012
סימול                 ArZa-001-001-013
Name: 990049383680205171, dtype: object

In [59]:
# df_repairs["first date"] = ''
# df_repairs["last date"] = ''

for index, row in df_repairs.iterrows():
    if index in list(df_Architect.index):
        try:
            print(index, type(index), index in list(df_Architect.index))
            print(f'df_repairs.loc[index, "תאריך מנורמל מוקדם"]: {df_repairs.loc[index, "תאריך מנורמל מוקדם"]}')
            df_repairs.loc[index, "תאריך מנורמל מוקדם"] = df_Architect.loc[index, "תאריך מנורמל מוקדם"]
            df_repairs.loc[index, "תאריך מנורמל מאוחר"] = df_Architect.loc[index, "תאריך מנורמל מאוחר"]
            
        except Exception as e:
            print(f'[error]: {e}')
            continue
    

990049383680205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049383690205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049383700205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049383710205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049383720205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049383730205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1998-06-01
990049383740205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1998-06-01
990049383750205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1998-06-01
990049383760205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049383770205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049383780205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
99004938379020

990050519690205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1955-07-28
990050519880205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1975-07-10
990050519890205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1975-07-10
990050519900205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1975-11-03
997010011510405171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1956
997010011510505171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1954
997010011510605171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1952
997010011510705171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1960-11-02
997010011510805171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1956
997010011510905171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1954
997010011511005171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1950-03

997007616423105171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1970
997007616423205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1975-11-14
997007616423305171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1970
997007616423405171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1975
997007616423505171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1970
997007616423605171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1975
997007616423705171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1960
997007616423805171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1970
997007616423905171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1960
997007616424005171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1960
997007616424105171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1960
997007616424205171 <class 

997008653590205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1989-04-19
997008653590305171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1955
997008653590405171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1988
997008653590505171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 2000
997008884354305171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1955
997008884354405171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1979-07-02
997008884354505171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1986-02-15
997008884354605171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1979-08-25
997008884354705171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 2000
997008884354805171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 2000
997008884354905171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 2000
99700888

990049340570205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1945
990049340580205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1962
990049340590205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1962
990049340600205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1969
990049340610205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1945
990049340620205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1945
990049340630205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1945
990049340640205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1945
990049340650205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1945
990049340660205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1945
990049340670205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1969
990049340680205171 <class 'str'>

...

990049390010205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049390020205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049390030205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049390040205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049390050205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049390060205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049390070205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049390080205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049390090205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049390100205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049390110205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049390120205171 <class 'str'>

997008521382105171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1936
997008521382205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1957
997008521382305171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1936
997008521382405171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1966
997008521382505171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1936
997008521382605171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1951-09-04
997008521382705171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1948-06-09
997008521382805171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1959-08-05
997008521382905171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1946
997008521383005171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1959-04-02
997008521383105171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1936
99700852

990049392330205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049392340205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049392350205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049392360205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1994-04-01
990049392370205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049392380205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049392390205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049392400205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049392410205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049392440205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1991
990049392450205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1935
990049392460205171 <class 

990048818220205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1940
990048818230205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1920
990048818250205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1939
990048818260205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1940
990048818270205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1965
990048818280205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1965
990048818290205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1965
990048818300205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1940
990048818360205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1942
990048818370205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1950
990048818380205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1955
990048818390205171 <class 'str'>

990050524820205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1968-11
990050524830205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1968
990050524850205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1967-11-19
990050524860205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1967-11-19
990050524870205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1966-02
990050524880205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1969-03
990050524890205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1967-06
990050524900205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1966-02-23
990050524910205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1960
990050524930205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1960
990050524940205171 <class 'str'> True
df_repairs.loc[index, "תאריך מנורמל מוקדם"]: 1974-01

In [60]:
df_repairs.head()

,245$a,520$a,655$a,כנראה לפתיחה תוכניות ושרטוטים מדיניות חדשה,093$a,093$c,093$d,פרטיות,952$f,008$7:4,...,597$a,597$b,999$a,906$a,907$i,907$c,907$k.1,700,תאריך מנורמל מוקדם,תאריך מנורמל מאוחר
MMS id,,,,,,,,,,,,,,,,,,,,,
990049383680205171,"תצלום תכנית - בי""ח מאיר 3 מתוך 16.",תצלום - תכנית קומה ראשונה של בית החולים מאיר,copies (documents);photographs,כן,nan,ArZa-001-001-013,ארכיון זרחי אדריכלים,nan,אין מגבלת פרטיות,1935,...,ארכיון זרחי אדריכלים זמין דיגיטלית באדיבות המש...,nan,PHOTOGRAPH;ARCHIVE,VIS;DIG,2,IE52225608,AR_IP_RANGE,nan,1935,2012
990049383690205171,"תצלום תכנית - בי""ח מאיר 4 מתוך 16.",תצלום - תכנית קומה שנייה של בית החולים מאיר,copies (documents);photographs,כן,nan,ArZa-001-001-014,ארכיון זרחי אדריכלים,nan,אין מגבלת פרטיות,1935,...,ארכיון זרחי אדריכלים זמין דיגיטלית באדיבות המש...,nan,PHOTOGRAPH;ARCHIVE,VIS;DIG,2,IE52225596,AR_IP_RANGE,nan,1935,2012
990049383700205171,"תצלום תכנית - בי""ח מאיר 5 מתוך 16.",תצלום - תכנית קומה שלישית של בית החולים מאיר,copies (documents);photographs,כן,nan,ArZa-001-001-015,ארכיון זרחי אדריכלים,nan,אין מגבלת פרטיות,1935,...,ארכיון זרחי אדריכלים זמין דיגיטלית באדיבות המש...,nan,PHOTOGRAPH;ARCHIVE,VIS;DIG,2,IE52225586,AR_IP_RANGE,nan,1935,2012
990049383710205171,"תצלום תכנית - בי""ח מאיר 6 מתוך 16.",תצלום - תכנית קומה רביעית של בית החולים מאיר,copies (documents);photographs,כן,nan,ArZa-001-001-016,ארכיון זרחי אדריכלים,nan,אין מגבלת פרטיות,1935,...,ארכיון זרחי אדריכלים זמין דיגיטלית באדיבות המש...,nan,PHOTOGRAPH;ARCHIVE,VIS;DIG,2,IE52225581,AR_IP_RANGE,1\$aלא ידוע$9heb$eצלם$8PreferredLanguageHeading,1935,2012
990049383720205171,"תצלום תכנית - בי""ח מאיר 7 מתוך 16.",תצלום - תכנית קומה חמישית של בית החולים מאיר,copies (documents);photographs,כן,nan,ArZa-001-001-017,ארכיון זרחי אדריכלים,nan,אין מגבלת פרטיות,1935,...,ארכיון זרחי אדריכלים זמין דיגיטלית באדיבות המש...,nan,PHOTOGRAPH;ARCHIVE,VIS;DIG,2,IE52225589,AR_IP_RANGE,nan,1935,2012


In [65]:
import numpy as np
df_repairs = df_repairs.replace(np.nan, "")
df_repairs = df_repairs.replace("nan", "")
df_repairs.to_excel("copyright_repairs_results_18102021_1.xlsx")

# Find Keywords

In [ ]:
from VC_collections import files

In [ ]:
keywords_spreadsheet = client.open_by_url(
            "https://docs.google.com/spreadsheets/d/1OeykES6h-vzNsdqtvn_TznyxY3rg5aoYEFahj6tVXBE"
        )

In [ ]:
df_terms, cols = files.create_df_from_gs(keywords_spreadsheet, "נושאים")

In [ ]:
terms_to_search = list(filter(None,df_terms["נושא רמה 3"].tolist()))
# terms_to_search = map()

In [ ]:
df_rei = pd.ExcelFile(r'Data/REI.xlsx').parse('Sheet1')

In [ ]:
df_rei.head()

In [ ]:
columns_to_save = ["סימול",
                   "רמת תיאור",
                   "כותרת",
                   "תיאור",
                   "תאריך חופשי",
                   "תאריך מנורמל מוקדם",
                   "תאריך מנורמל מאוחר",
                   "יוצרים",
                   "מילות מפתח - אישים",
                   "מילות מפתח - מוסדות",
                   "מילות מפתח_נושאים",
                   "מילות מפתח_מקומות"]

In [ ]:
from collections import defaultdict
dfs = defaultdict()
for term in terms_to_search:

    df = master_catalog[master_catalog.apply(lambda row: row.astype(str).str.contains(term).any(), axis=1)]
    dfs[term]= df[columns_to_save]
    

In [ ]:
from collections import defaultdict
dfs = defaultdict()

for term in terms_to_search:
    df = df_rei[df_rei.apply(lambda row: row.astype(str).str.contains(term).any(), axis=1)]
    dfs[term] = df

In [ ]:
occurences = dict()

for key in dfs.keys():
    occurences[key] = len(dfs[key])
    print(f"{key} - מספר מופעים: {len(dfs[key])}")



In [ ]:
df_terms

In [ ]:
df_occurances = pd.DataFrame(occurences, index=["occurances"]).transpose()
df_terms = df_terms.set_index("נושא רמה 3")
# for index, row in df_occurances.iterrows():
#     df_


In [ ]:
df_join = df_occurances.join(df_terms)
df_join.loc[:,["נושא רמה 1", "נושא רמה 2", "occurances"]].to_excel(input("name:")+".xlsx")

In [ ]:

keywords_spreadsheet = pd.ExcelWriter(input(str("enter file name without extension"))+'.xlsx')
for term, df in dfs.items():
    print(term)
    print(len(df))
    df.to_excel(keywords_spreadsheet, sheet_name=term.decode('utf-8','ignore').encode("utf-8"))

In [ ]:
from pandas import ExcelWriter


def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for term, df in list_dfs.items():
            df.to_excel(writer,f's_{term}')
        writer.save()

In [ ]:
save_xls(dfs, "keywords.xlsx")

# update correct names
from Python\Projects\Authorities

In [ ]:
import json

with open("../Projects/Authorities/names_variant.json", encoding='utf-8') as fp:
    names_dict = json.load(fp)

In [ ]:
dfs = [df_Architect, df_Dance, df_Design, df_Theater]


In [ ]:
from VC_collections.authorities import convert_dict

names_mapping_dict = convert_dict(names_dict)
names_mapping_dict = {key: value[0] for key, value in names_mapping_dict.items()} 

In [ ]:
%%timeit

for key in names_mapping_dict.keys():
    for df in dfs:
        df.replace(key, names_mapping_dict[key])

In [ ]:
from  df2gspread import df2gspread as d2g
from VC_collections.Collection import get_google_drive_credentials

In [ ]:
def update_master_catalog_gfile(df, gfile, name, credentials):
    wkt = d2g.upload(df, 
                     gfile=gfile,
                     wks_name=name,
                    credentials=credentials, 
                    )
    print( f'uploaded {eval(df)} to google sheets')

In [ ]:
cred = get_google_drive_credentials()


In [ ]:
update_master_catalog_gfile(df_Architect, catalog_files["Architect"], "test", credentials=cred)
update_master_catalog_gfile(df_Dance, catalog_files["Dance"], "test", credentials=cred)
update_master_catalog_gfile(df_Design, catalog_files["Design"], "test", credentials=cred)
update_master_catalog_gfile(df_Theater, catalog_files["Theater"], "test", credentials=cred)

## Look up creators in entire catalog

In [ ]:
import numpy as np

term_to_search = input(str("What to search? "))

mask = np.column_stack([master_catalog[col].astype(str).str.contains(term_to_search, na=False) for col in master_catalog])
df_test = master_catalog.loc[mask.any(axis=1)]

In [ ]:
df_test[['יוצרים', "ארכיון"]]

In [ ]:
import xmltodict 
import json

In [ ]:
with open(r"C:\Users\Yaelg\Google Drive\National_Library\Python\VC-Architect\PYeIl\Data\processed\PYeIl_final_20210617.xml", encoding="utf8") as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

In [ ]:
xml_file.close()


In [ ]:
json_data = json.dumps(data_dict)

In [ ]:
with open("data.json", "w") as json_file:
        json_file.write(json_data)

In [ ]:
json_file.close()
